<a href="https://colab.research.google.com/github/abdessamed122/1-st-projet-with-API/blob/main/YouTube%20Video%20Chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:

!pip install groq pytube youtube-transcript-api
!pip install yt_dlp
!pip install langchain faiss-cpu sentence-transformers



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [13]:
!pip install langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.49
    Uninstalling langchain-core-0.3.49:
      Successfully uninstalled langchain-core-0.3.49
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.22
    Uninstalling langchain-0.3.22:
      Successfully uninstalled langchain-0.3.22


In [16]:
!pip install langchain-groq # Install the langchain-groq package, which contains the ChatGroq class.

In [53]:
   GROQ_API_KEY = "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"
   # Replace with your API key

In [54]:
import re
from youtube_transcript_api import YouTubeTranscriptApi

# استخدام واجهة YouTubeTranscriptAPI لجلب الترجمة
def get_transcript(video_id):
    transcript = YouTubeTranscriptApi.get_transcript(video_id)
      # transcript = YouTubeTranscriptApi.get_transcript(video_id)

    return transcript


In [55]:
# دمج النصوص بحيث يحتوي كل نص على 60 كلمة مع الحفاظ على التوقيت
def merge_texts_to_60_words(transcript, max_words_per_text=90):
    merged_texts = []
    current_text = []
    current_word_count = 0
    current_start_time = None

    for entry in transcript:
        text = entry['text']
        words = text.split()

        for word in words:
            if current_word_count == 0:
                current_start_time = entry['start']  # تحديد وقت البدء لأول كلمة من النص
            current_text.append(word)
            current_word_count += 1

            # إذا وصلنا إلى 60 كلمة، نقوم بإضافة النص المدمج
            if current_word_count >= max_words_per_text:
                merged_texts.append((current_start_time, " ".join(current_text)))
                current_text = []  # إعادة تعيين النص المدمج
                current_word_count = 0

    # إذا تبقى نص غير مكتمل يحتوي على أقل من 60 كلمة، نضيفه أيضًا
    if current_text:
        merged_texts.append((current_start_time, " ".join(current_text)))

    return merged_texts

In [95]:


# تنسيق الترجمة مع التوقيت
def format_transcript_with_time(transcript):
    formatted_transcript = []

    merged_texts = merge_texts_to_60_words(transcript)

    for start_time, text in merged_texts:
        # تحويل الوقت إلى صيغة (00:00:00)
        minutes, seconds = divmod(start_time, 60)
        hours, minutes = divmod(minutes, 60)
        formatted_time = f"{int(hours):02}:{int(minutes):02}:{int(seconds):02}"

        # إضافة التوقيت والنص
        formatted_transcript.append(f"{formatted_time}\n{text}")

    return "\n\n".join(formatted_transcript)

# جلب الترجمة من الفيديو
video_id = '3u6QO3lKOtA'  # أدخل هنا الـ video ID
transcript = get_transcript(video_id)

# تنسيق الترجمة مع التوقيت
formatted_transcript = format_transcript_with_time(transcript)
# print(formatted_transcript)

In [96]:
from langchain.text_splitter import CharacterTextSplitter
from typing import List, Dict, Tuple

def get_text_chunks_from_transcript(
    formatted_transcript: str,
    video_id: str,
    chunk_size: int = 1000,
    chunk_overlap: int = 200
) -> Tuple[List[str], List[Dict[str, str]]]:
    """
    Split formatted transcript text (with timestamps) into chunks for embedding.

    Args:
        formatted_transcript: Transcript text with timestamps.
        video_id: YouTube Video ID.
        chunk_size: Maximum chunk size.
        chunk_overlap: Overlap between chunks.

    Returns:
        chunks: List of text chunks.
        metadata: List of dicts with video_id and chunk number.
    """
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )

    chunks = text_splitter.split_text(formatted_transcript)

    metadata = [
        {"video_id": video_id, "chunk": i + 1}
        for i in range(len(chunks))
    ]

    return chunks, metadata

get_text_chunks= get_text_chunks_from_transcript(
    formatted_transcript,
    video_id,
    chunk_size = 1000,
    chunk_overlap = 200
)
# print(get_text_chunks)
# print(metadata)  # Prints metadata for each chunk
chunks, metadata = get_text_chunks_from_transcript(formatted_transcript, video_id)
print(chunks)  # Prints the text chunks
print(metadata)  # Prints metadata for each chunk


["00:00:00\nO Abu Hazem, why do we hate death? You like the video, don't forget to click on the button Like and subscribe to the channel, oh Sheikh Meshari human when he enters the world, he enters it by one When he comes out of it, he comes out when he is ran in His grave is an individual and when he stands in the hands of his Lord He stands one, so he should mention this Lord, glory be to Him He says that to decorate what is between him\n00:00:25\nand between Even if a day comes to him, he will not come Looking is like the servant who kept an individual to His money is in response and dragged to him, but he comes with who His relationship with him is good and his ropes with him, and all of them It came on the day of resurrection, as our Lord said in Another place, and you have come to us as an individual as we created you The first time and you left what we have gave you\n00:00:56", '00:00:56\nbehind your back We have made you money, supporters and servants And all of this left you b

In [97]:
from typing import List, Dict, Union
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

def get_vectorstore(
    text_chunks: List[str],
    metadata: List[Dict[str, str]],
    embedding_model: Union[str, HuggingFaceEmbeddings],
    save_path: str = None
) -> FAISS:
    """
    Create a FAISS vectorstore from text chunks & metadata.

    Args:
        text_chunks: List of text chunks.
        metadata: Corresponding metadata for each chunk.
        embedding_model: Model name (str) or HuggingFaceEmbeddings object.
        save_path: Optional path to persist FAISS vectorstore.

    Returns:
        FAISS Vectorstore object.
    """

    embeddings = embedding_model if isinstance(embedding_model, HuggingFaceEmbeddings) else HuggingFaceEmbeddings(model_name=embedding_model)

    vectorstore = FAISS.from_texts(
        texts=text_chunks,
        embedding=embeddings,
        metadatas=metadata
    )

    if save_path:
        vectorstore.save_local(save_path)

    return vectorstore
vectorstore = get_vectorstore(chunks, metadata, embedding_model=HuggingFaceEmbeddings(), save_path="faiss_index")


<ipython-input-97-1613355cc9be>:36: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  vectorstore = get_vectorstore(chunks, metadata, embedding_model=HuggingFaceEmbeddings(), save_path="faiss_index")


In [98]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain_groq import ChatGroq
from langchain.vectorstores import FAISS

def get_conversation_chain(vectorstore: FAISS, llm_model: str = "llama-3.3-70b-specdec"):
    """
    Initialize a conversational retrieval chain with LLM and vectorstore.

    Args:
        vectorstore: FAISS vectorstore.
        llm_model: Name of LLM model to use (OpenAI or Groq).

    Returns:
        ConversationalRetrievalChain object.
    """
    groq_api_key="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"
    # Dynamically choose LLM
    if llm_model.startswith("llama"):
        llm = ChatGroq(model=llm_model,groq_api_key=groq_api_key)

    # else:
    #     llm = ChatOpenAI(model=llm_model)

    memory = ConversationBufferMemory(
        memory_key='chat_history',
        return_messages=True
    )

    prompt_template = PromptTemplate(
        input_variables=["question", "context"],
        template="""
You are an intelligent assistant.

Use ONLY the context provided to answer the question.

If the answer is not in the context, say "Sorry, I don't know."

Answer in a concise and clear way. Provide source when possible.

Context:
{context}

Question:
{question}

Answer (with source if available):
"""
    )

    chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory,
        combine_docs_chain_kwargs={"prompt": prompt_template},
        return_source_documents=True
    )

    return chain



In [99]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain_groq import ChatGroq
from langchain.vectorstores import FAISS

def get_conversation_chain(vectorstore: FAISS, llm_model: str = "llama-3.3-70b-specdec"):
    """
    Initialize a conversational retrieval chain with LLM and vectorstore.

    Args:
        vectorstore: FAISS vectorstore.
        llm_model: Name of LLM model to use (OpenAI or Groq).

    Returns:
        ConversationalRetrievalChain object.
    """
    groq_api_key="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"
    # Dynamically choose LLM
    if llm_model.startswith("llama"):
        llm = ChatGroq(model=llm_model,groq_api_key=groq_api_key)

    # else:
    #     llm = ChatOpenAI(model=llm_model)

    memory = ConversationBufferMemory(
        memory_key='chat_history',
        return_messages=True,
        # Explicitly setting the output_key
        output_key='answer'
    )

    prompt_template = PromptTemplate(
        input_variables=["question", "context"],
        template="""
You are an intelligent assistant.

Use ONLY the context provided to answer the question.

If the answer is not in the context, say "Sorry, I don't know."

Answer in a concise and clear way.

Context:
{context}

Question:
{question}

"""
    )

    chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory,
        combine_docs_chain_kwargs={"prompt": prompt_template},
        return_source_documents=True
    )

    return chain

In [100]:
conversation_chain = get_conversation_chain(vectorstore, llm_model="llama-3.3-70b-specdec")


In [101]:
# After getting the response from the conversation chain
response = conversation_chain({
    'question': 'ما هو الموضوع الرئيسي للفيديو؟'
})

# Extract the answer
answer = response['answer']
print(answer)


الموضوع الرئيسي للفيديو هو الحديث عن الموت واليوم الآخر والحث على التذكر والتقرب إلى الله.


In [102]:
# After getting the response from the conversation chain
response = conversation_chain({
    'question': 'أعطني بعض الأسئلة والأجوبة'

})

# Extract the answer
answer = response['answer']
print(answer)

الفيديو يحتوي على تعاليم دينية ويتناول مواضيع متعلقة بالحياة الآخرة والخلود. بعض الأسئلة والأجوبة التي يمكن أن تناقشها حول الفيديو تشمل:

- ما هو مفهوم الجنة والنار في الإسلام؟
- كيف يمكننا تحقيق الفوز في الحياة الآخرة؟
- ما هي الأعمال الصالحة التي يمكننا القيام بها لتصل إلى الجنة؟
- كيف يمكننا تجنب الأعمال السيئة التي قد تؤدي إلى النار؟
- ما هي دورة الحساب في يوم القيامة؟
- كيف يمكننا الاستعداد للقاء الله في الحياة الآخرة؟


In [105]:
# After getting the response from the conversation chain
response = conversation_chain({
    'question': ' اجبني على هده الاسئلة  '

})

# Extract the answer
answer = response['answer']
print(answer)

إجابات على الأسئلة:
- مفهوم الجنة والنار في الإسلام: الجنة هي مكان النعيم والسرور للأذين أطاعوا الله، والنار هي مكان العذاب للأذين عصوا الله.
- تحقيق الفوز في الحياة الآخرة: يمكننا تحقيق الفوز من خلال الأعمال الصالحة والطاعة لله.
- الأعمال الصالحة: يمكننا القيام بالصلوات، والصدقة، والصيام، والحج، والقراءة للقرآن.
- تجنب الأعمال السيئة: يمكننا تجنب الأعمال السيئة من خلال الابتعاد عن الشهوات والرذائل.
- دورة الحساب في يوم القيامة: الحساب هو عملية محاسبة الأفراد على أعمالهم في الحياة الدنيا.
- الاستعداد للقاء الله: يمكننا الاستعداد من خلال التوبة، والتقرب إلى الله، والقيام بالأعمال الصالحة.
